# Kaggle Deep Learning Overview

This notebook is for notes and exercise from the Kaggle Learning 2.0 Deep Learning Course.

1. [Introduction to Convolutions for Computer Vision Based DL](#conv)  
2. [Transfer Learning](#transfer)
3.  
4.  

<a id='conv'></a>
## Introduction to Convolutions for Computer Vision Based DL

Convolutions are a way for a computer to 'determine' information about an image. These convolutions are an array of values learned by deep learning algorithms that, when applied (mulitplied) to pixel values, can determine what that area/pixel is 'supposed' to be in a way. 

For example, given a 2x2 array, a [ [1.5,1.5], [-1.5,-1.5] ] convolution can determine if a square of 4 pixels (shaded from white-black) makes up a horizontal line.

Let's do another example. The below cell will print a raw image as well as the output from applying a basic convolution to the image.


In [ ]:
%reload_ext autoreload
%autoreload 2
from my_modules import dl_helpers as helpers
from termcolor import cprint # colored prints

horizontal_line_conv = [[ 1, 1],
                        [-1,-1]]
vert_line_conv = [[1, -1],
                  [1, -1]]

# load_my_image and visualize_conv are utility functions taken from the Kaggle/learntools repository
original_image = helpers.load_my_image('dog_1')
helpers.visualize_conv(original_image, horizontal_line_conv, 'Horizontal Line Conv')
helpers.visualize_conv(original_image, vert_line_conv, 'Vertical Line Conv')

## Convolutions and tensorflow
The below sections introduce and show us how to setup our environment for using convolutions, keras, and tensorflow.

#### How convolutions work with DL
The video below explains the basics of how convolutions work to 'learn' image sets and predict.  
https://youtu.be/ToBPiUlLFEY

#### How to setup code for convolution DL
The video below explains some of the basic setup for a pre-existing model and convolutions.  
https://youtu.be/sDG5tPtsbSA

### Image Setup


In [ ]:
# Choose images to work with for our example
from os.path import join

image_dir = './images/dogs/train/'
img_dict = {'dog_1' : "00a338a92e4e7bf543340dc849230e75.jpg",
            'dog_2' : "0b345d4f2434903c374ad8b8513a289b.jpg",
            'dog_3' : "0db44ddb42bf1f97de987abe2bf01839.jpg",
            'dog_4' : "01f8540fb1084107a6eb3e528f82c1aa.jpg"}

img_paths = [join(image_dir, file) for file in img_dict.values()]
img_paths

### Function to Read and Prep Images for Modeling

In [ ]:
import numpy as np
# conda install -c conda-forge tensorflow
from tensorflow.python.keras.applications.resnet import preprocess_input
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
image_size = 224

def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return output

### Create Model with Pre-Trained Weights

In [ ]:
# conda install -c conda-forge keras
from tensorflow.keras.applications import ResNet50

my_model = ResNet50(weights='./pre-trained/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
test_data = read_and_prep_images(img_paths)
preds = my_model.predict(test_data)

### Visualize Predictions


In [ ]:
from my_modules.dl_helpers import decode_predictions
from IPython.display import Image, display

most_likely_labels = decode_predictions(preds, top=3)
for i, img_path in enumerate(img_paths):
    display(Image(img_path))
    cprint(f'Most Likely Label: {i + 1}', 'green')
    print(most_likely_labels[i])

<a id='transfer'></a>
## Transfer Learning
Transfer learning is a technique of taking an existing pre-trained model and cutting off the output layer so we can quickly and effiently apply the model to other, maybe more specific, problems. An example of this is taking our previously trained model and using it to classify whether an image looks 'urban' or 'rural'.

In our personal example, we'll be determining whether a photo is landscape or portrait. We'll be using a dataset of dog photos which are either horizontal or vertical. The below code will be very similar to the Kaggle example.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = './pre-trained/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Setup a new Sequential model
my_new_model = Sequential()
# Add the original pre-trained data minus the top (output) layer)
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
# Add our own (Dense) output layer with our 2 classes and softmax activation (probabilities)
my_new_model.add(Dense(num_classes, activation='softmax'))

# Disable the first (pre-trained) layer from training
my_new_model.layers[0].trainable = False



In [ ]:
# Compile the model
my_new_model.compile(optimize='sgd',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

### Compile Step Review
There were three arguments supplied to the compile step, let's review them:
- **Optimizer** determines how we determine the numerical values that make up the model. It can affect the resulting model and its predictions. 
- **Loss** determines what goal we optimize when determining numerical values in the model. It can affect the resulting model and predictions.
- **Metrics** determines only what we print out while the model is being build, but it doesn't affect the model itself.

### Fitting the Model
We have our training and validation sets in ```'./sideways_dogs/images/train``` and ```'./sideways_dogs/images/val```. These will be used when setting up ```train_generator``` and ```validation_generator```.

We have 220 training images and 217 validation images. We'll use a batch size of 10 and have 22 epochs.

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# our image size in px
image_size = 244

data_generator = ImageDataGenerator(preprocess_input)

train_path = './sideways_dogs/images/train'
val_path = './sideways_dogs/images/val'

train_generator = data_generator.flow_from_directory(
                                    directory=train_path,
                                    target_size=(image_size, image_size),
                                    batch_size=10,
                                    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
                                        directory=val_path,
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

fit_stats = my_new_model.fit_generator(train_generator,
                                       steps_per_epoch=22,
                                       validation_data=validation_generator,
                                       validation_steps=1)

## 3. Data Augmentation
Data augmentation is a good technique to use when we have a limited amount of data. Augmentation is basically taking original photos and manipulating them to give us more data. For example, take an urban photo. Flipping the photo on the horizontal will give us a completely different photo that's still an urban photo! This essentially doubles our data if we flip all the images.

**Note**: While working with augmentation, we still need an un-augmented generator for our validation set. Only the training set should be augmented.

In [ ]:
# reimport for fun
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# our image size in px
image_size = 244

data_generator_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    horizontal_flip = True,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1)
data_gen_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

train_path = './sideways_dogs/images/train'
val_path = './sideways_dogs/images/val'

aug_train_generator = data_generator_aug.flow_from_directory(
                                    directory=train_path,
                                    target_size=(image_size, image_size),
                                    batch_size=12,
                                    class_mode='categorical')

validation_generator = data_gen_no_aug.flow_from_directory(
                                        directory=val_path,
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

fit_stats_aug = my_new_model.fit_generator(aug_train_generator,
                                       steps_per_epoch=22,
                                       validation_data=validation_generator,
                                       validation_steps=1)


## 4. Model from Scratch
Now let's take everything we learned above, and guidance by our friend Kaggle and build our own model. We'll use the fashion-mnist set that tries to categorize clothing based on low res images.

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

# basically the size of our images
img_rows, img_cols = 28, 28
# num of categorical classes
num_classes = 10

def prep_data(raw):
    """
    Prepares fashion-mnist data 
    params:
        raw: the raw image pixel data
    returns:
        out_x: predictors dataframe
        out_y: target dataframe
    """
    # get the first column (the target labels)
    y = raw[:, 0]
    # one_hot_encode the target labels
    out_y = keras.utils.to_categorical(y, num_classes)
    # get the predictors (everything except first row)
    x = raw[:,1:]
    num_images = raw.shape[0] # get total num of images through shape of the first* row
    # original data has each pixel seperated into different columns
    #  reshape so each row is back to 28x28 image shape
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    # normalize the pixel data to 0~1 values
    out_x = out_x / 255
    return out_x, out_y

fashion_file = "./clothes/fashion-mnist_train.csv"
# load the training data
fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(fashion_data)

In [4]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D

# get base sequential model ready
fashion_model = Sequential()
# add first (input) layer 
fashion_model.add(Conv2D(12,
                         kernel_size=(3,3),
                         activation='relu',
                         input_shape=(img_rows, img_cols, 1)))
# add more convolution layers
fashion_model.add(Conv2D(20,
                         kernel_size=(3,3),
                         activation='relu'))
fashion_model.add(Conv2D(20,
                         kernel_size=(3,3),
                         activation='relu'))
# Flatten into a singular dimension (aka 2d -> 1d)
fashion_model.add(Flatten())
# add an extra dense layer for good measures
fashion_model.add(Dense(100, activation='relu'))
# output/prediction layer
fashion_model.add(Dense(num_classes,
                        activation='softmax'))


In [5]:
# compile dat model
fashion_model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer='adam',
                      metrics=['accuracy'])

In [6]:
# fit it 
fashion_model.fit(x, y, batch_size = 100, epochs=4, validation_split=0.2)


Train on 48000 samples, validate on 12000 samples
Epoch 1/4
48000/48000 [==============================] - 65s 1ms/sample - loss: 0.4887 - accuracy: 0.8227 - val_loss: 0.3643 - val_accuracy: 0.8738
Epoch 2/4
48000/48000 [==============================] - 62s 1ms/sample - loss: 0.3169 - accuracy: 0.8854 - val_loss: 0.3172 - val_accuracy: 0.8897
Epoch 3/4
48000/48000 [==============================] - 58s 1ms/sample - loss: 0.2628 - accuracy: 0.9041 - val_loss: 0.2730 - val_accuracy: 0.9028
Epoch 4/4
48000/48000 [==============================] - 59s 1ms/sample - loss: 0.2237 - accuracy: 0.9168 - val_loss: 0.2616 - val_accuracy: 0.9082


Our model gave us a score of **90.82%**. Now let's train a second model to compare different parameters and see what we get.

In [7]:
second_fashion_model = Sequential()
second_fashion_model.add(Conv2D(9,
                         kernel_size=(3,3),
                         activation='relu',
                         input_shape=(img_rows, img_cols, 1)))
second_fashion_model.add(Conv2D(18,
                         kernel_size=(3,3),
                         activation='relu'))
second_fashion_model.add(Conv2D(36,
                         kernel_size=(3,3),
                         activation='relu'))
second_fashion_model.add(Conv2D(18,
                         kernel_size=(3,3),
                         activation='relu'))
second_fashion_model.add(Flatten())
second_fashion_model.add(Dense(90, activation='relu'))
second_fashion_model.add(Dense(num_classes, activation='softmax'))
second_fashion_model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer='adam',
                      metrics=['accuracy'])
second_fashion_model.fit(x, y, batch_size = 100, epochs=3, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 62s 1ms/sample - loss: 0.5065 - accuracy: 0.8161 - val_loss: 0.3601 - val_accuracy: 0.8726
Epoch 2/3
48000/48000 [==============================] - 60s 1ms/sample - loss: 0.3112 - accuracy: 0.8867 - val_loss: 0.3041 - val_accuracy: 0.8966
Epoch 3/3
48000/48000 [==============================] - 61s 1ms/sample - loss: 0.2536 - accuracy: 0.9055 - val_loss: 0.2645 - val_accuracy: 0.9087


And our second model gave us a score of **90.87%**, just slightly better. 

In [9]:
fashion_test_file = "./clothes/fashion-mnist_test.csv"
# load the training data
fashion_test_data = np.loadtxt(fashion_test_file, skiprows=1, delimiter=',')
test_x, test_y = prep_data(fashion_test_data)
second_fashion_model.predict(test_x, batch_size = 100)

array([[9.81861830e-01, 2.59648778e-06, 5.61968598e-04, ...,
        4.79698599e-07, 4.95854183e-05, 6.04083027e-07],
       [6.15868748e-06, 9.99985218e-01, 2.77741123e-07, ...,
        3.87616694e-09, 3.79854583e-07, 1.00034974e-07],
       [1.45539064e-02, 1.63495915e-05, 7.19616055e-01, ...,
        1.82456515e-05, 3.89483394e-05, 9.14063712e-05],
       ...,
       [1.70235071e-05, 1.41697853e-07, 1.33747426e-05, ...,
        1.97756861e-04, 9.99737561e-01, 3.31930733e-06],
       [6.81694038e-03, 1.17161699e-05, 6.94141956e-04, ...,
        5.14560525e-05, 9.84435618e-01, 1.26561907e-04],
       [2.89761946e-02, 7.77235627e-01, 1.25577589e-02, ...,
        1.75265614e-05, 1.65918898e-02, 8.47225529e-05]], dtype=float32)

TODO: What do we do with the numby array of predictions